# 📈 Sales Forecasting Project with Mandatory Bonus Features

## Walmart Sales Forecast Implementation

This notebook implements a comprehensive sales forecasting system with:
- ✅ Time-based feature engineering
- ✅ Lag features and rolling averages (bonus)
- ✅ Seasonal decomposition (bonus)
- ✅ Multiple ML models including XGBoost and LightGBM (bonus)
- ✅ Time-aware cross-validation (bonus)
- ✅ Comprehensive visualizations


In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Machine Learning imports
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

# Time series analysis
from statsmodels.tsa.seasonal import seasonal_decompose

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("📚 Libraries imported successfully!")


## Step 1: Dataset Setup

For this demonstration, we'll create a synthetic dataset that mimics the Walmart sales structure. In a real scenario, you would download the actual Walmart dataset from Kaggle.


In [ ]:
# Create synthetic Walmart-like dataset
print("📊 Creating synthetic Walmart-like dataset...")

np.random.seed(42)
n_stores = 5
n_depts = 3
n_weeks = 150

data = []
for store in range(1, n_stores + 1):
    for dept in range(1, n_depts + 1):
        for week in range(n_weeks):
            date = pd.date_range('2020-01-01', periods=n_weeks, freq='W')[week]
            
            # Create realistic sales pattern with seasonality
            base_sales = 10000 + store * 2000 + dept * 1000
            seasonal = 2000 * np.sin(2 * np.pi * week / 52)  # Yearly seasonality
            trend = week * 10  # Slight upward trend
            noise = np.random.normal(0, 1000)
            
            weekly_sales = max(0, base_sales + seasonal + trend + noise)
            
            # Add holiday effects
            is_holiday = (week % 52 in [45, 46, 47, 48]) or (week % 52 in [0, 1, 2, 3])
            if is_holiday:
                weekly_sales *= 1.3
            
            data.append({
                'Store': store,
                'Dept': dept,
                'Date': date,
                'Weekly_Sales': weekly_sales,
                'IsHoliday': is_holiday,
                'Size': np.random.choice([10000, 20000, 30000]),
                'Type': np.random.choice(['A', 'B', 'C']),
                'Temperature': np.random.normal(70, 20),
                'Fuel_Price': np.random.normal(3.5, 0.5),
                'CPI': np.random.normal(200, 20),
                'Unemployment': np.random.normal(8, 2)
            })

df = pd.DataFrame(data)
print(f"✅ Synthetic dataset created: {df.shape}")
print(f"Date range: {df['Date'].min()} to {df['Date'].max()}")
df.head()


## Step 2: Data Preprocessing

Convert dates, sort data, and handle missing values.


In [ ]:
# Convert Date column to datetime format
df['Date'] = pd.to_datetime(df['Date'])

# Sort data by date for each store/department
df = df.sort_values(by=['Store', 'Dept', 'Date']).reset_index(drop=True)

# Check for missing values
print("Missing values per column:")
missing_info = df.isnull().sum()
print(missing_info[missing_info > 0])

# Handle missing values
numeric_columns = df.select_dtypes(include=[np.number]).columns
for col in numeric_columns:
    if df[col].isnull().any():
        df[col].fillna(df[col].median(), inplace=True)

print(f"\nFinal dataset shape: {df.shape}")
print(f"Date range: {df['Date'].min()} to {df['Date'].max()}")


## Step 3: Feature Engineering (Core + Bonus Features)

This is the heart of the forecasting system. We'll create:
- Time-based features
- Lag features (previous sales)
- **Rolling averages (BONUS)**
- **Seasonal decomposition (BONUS)**
- Additional statistical features


In [ ]:
print("🔧 Creating comprehensive features...")

# Make a copy to avoid modifying original
df_features = df.copy()

# Time-based features
df_features['year'] = df_features['Date'].dt.year
df_features['month'] = df_features['Date'].dt.month
df_features['week'] = df_features['Date'].dt.isocalendar().week
df_features['dayofweek'] = df_features['Date'].dt.dayofweek
df_features['is_weekend'] = df_features['dayofweek'].isin([5, 6]).astype(int)
df_features['quarter'] = df_features['Date'].dt.quarter

print("✅ Time-based features created")


In [ ]:
# Lag features (previous sales)
df_features['lag_1'] = df_features.groupby(['Store', 'Dept'])['Weekly_Sales'].shift(1)
df_features['lag_2'] = df_features.groupby(['Store', 'Dept'])['Weekly_Sales'].shift(2)
df_features['lag_3'] = df_features.groupby(['Store', 'Dept'])['Weekly_Sales'].shift(3)
df_features['lag_4'] = df_features.groupby(['Store', 'Dept'])['Weekly_Sales'].shift(4)

print("✅ Lag features created")


In [ ]:
# Rolling averages (BONUS FEATURE)
df_features['rolling_mean_3'] = df_features.groupby(['Store', 'Dept'])['Weekly_Sales'].shift(1).rolling(window=3).mean()
df_features['rolling_mean_7'] = df_features.groupby(['Store', 'Dept'])['Weekly_Sales'].shift(1).rolling(window=7).mean()
df_features['rolling_std_3'] = df_features.groupby(['Store', 'Dept'])['Weekly_Sales'].shift(1).rolling(window=3).std()
df_features['rolling_std_7'] = df_features.groupby(['Store', 'Dept'])['Weekly_Sales'].shift(1).rolling(window=7).std()

# Rolling min/max
df_features['rolling_min_3'] = df_features.groupby(['Store', 'Dept'])['Weekly_Sales'].shift(1).rolling(window=3).min()
df_features['rolling_max_3'] = df_features.groupby(['Store', 'Dept'])['Weekly_Sales'].shift(1).rolling(window=3).max()

print("✅ Rolling statistical features created (BONUS)")


In [ ]:
# Seasonal decomposition (BONUS FEATURE)
print("🔧 Performing seasonal decomposition...")

seasonal_features = []

# Apply seasonal decomposition for each store-dept combination
for (store, dept), group in df_features.groupby(['Store', 'Dept']):
    if len(group) >= 52:  # Need at least 52 weeks for yearly seasonality
        try:
            # Perform seasonal decomposition
            result = seasonal_decompose(group['Weekly_Sales'], model='additive', period=52)
            
            # Add to dataframe
            group['trend'] = result.trend
            group['seasonal'] = result.seasonal
            group['residual'] = result.resid
            
            seasonal_features.append(group)
        except:
            # If decomposition fails, add zeros
            group['trend'] = 0
            group['seasonal'] = 0
            group['residual'] = 0
            seasonal_features.append(group)
    else:
        # Not enough data for seasonal decomposition
        group['trend'] = 0
        group['seasonal'] = 0
        group['residual'] = 0
        seasonal_features.append(group)

df_features = pd.concat(seasonal_features, ignore_index=True)
print("✅ Seasonal decomposition completed (BONUS)")


In [ ]:
# Additional features
df_features['sales_ratio'] = df_features['Weekly_Sales'] / df_features.groupby(['Store', 'Dept'])['Weekly_Sales'].transform('mean')
df_features['price_ratio'] = df_features['CPI'] / df_features.groupby(['Store'])['CPI'].transform('mean')

# Holiday features
df_features['is_holiday'] = df_features['IsHoliday'].astype(int)

# Store and department features
df_features['store_size_category'] = pd.cut(df_features['Size'], bins=3, labels=['Small', 'Medium', 'Large'])
df_features['store_size_category'] = df_features['store_size_category'].astype('category').cat.codes

# Type encoding
df_features['type_encoded'] = df_features['Type'].astype('category').cat.codes

# Remove rows with NaN values created by lag features
df_features = df_features.dropna()

print(f"✅ Additional features created")
print(f"Final features dataset shape: {df_features.shape}")
print(f"Features created: {len(df_features.columns) - len(df.columns)} new features")


## Step 4: Time-Aware Data Splitting

Split data chronologically (not randomly) to avoid data leakage.


In [ ]:
# Select features for modeling
feature_cols = [col for col in df_features.columns if col not in [
    'Date', 'Weekly_Sales', 'Store', 'Dept', 'Type', 'IsHoliday'
]]

X = df_features[feature_cols]
y = df_features['Weekly_Sales']

print(f"Features used: {len(feature_cols)}")
print(f"Feature columns: {feature_cols}")

# Time-aware split (80-20)
split_idx = int(len(df_features) * 0.8)
X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]
test_dates = df_features[split_idx:]['Date']

print(f"\nTraining set size: {len(X_train)}")
print(f"Test set size: {len(X_test)}")
print(f"Training date range: {df_features[:split_idx]['Date'].min()} to {df_features[:split_idx]['Date'].max()}")
print(f"Test date range: {test_dates.min()} to {test_dates.max()}")


## Step 5: Baseline Models

Train Linear Regression and Random Forest models.


In [ ]:
print("🤖 Training baseline models...")

models = {}

# Linear Regression
print("Training Linear Regression...")
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)
models['LinearRegression'] = lr_model

# Random Forest
print("Training Random Forest...")
rf_model = RandomForestRegressor(
    n_estimators=100,
    random_state=42,
    n_jobs=-1
)
rf_model.fit(X_train, y_train)
models['RandomForest'] = rf_model

print("✅ Baseline models trained!")


## Step 6: Advanced Models (BONUS FEATURES)

Train XGBoost and LightGBM models with advanced configurations.


In [ ]:
print("🚀 Training advanced models (BONUS)...")

# XGBoost (BONUS)
print("Training XGBoost...")
xgb_model = XGBRegressor(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    objective='reg:squarederror',
    random_state=42,
    n_jobs=-1
)
xgb_model.fit(X_train, y_train)
models['XGBoost'] = xgb_model

# LightGBM (BONUS)
print("Training LightGBM...")
lgb_model = LGBMRegressor(
    n_estimators=500,
    learning_rate=0.05,
    num_leaves=31,
    feature_fraction=0.8,
    random_state=42,
    n_jobs=-1,
    verbose=-1
)
lgb_model.fit(X_train, y_train)
models['LightGBM'] = lgb_model

print("✅ Advanced models trained!")


## Step 7: Time-Aware Cross-Validation (BONUS)

Perform time-aware cross-validation to properly evaluate time series models.


In [ ]:
def time_aware_cross_validation(X_train, y_train, model_name='XGBoost'):
    """
    Time-aware cross-validation (BONUS FEATURE)
    """
    print(f"🔄 Performing time-aware cross-validation for {model_name}...")
    
    # Initialize model
    if model_name == 'XGBoost':
        model = XGBRegressor(
            n_estimators=100,
            learning_rate=0.1,
            max_depth=6,
            random_state=42,
            n_jobs=-1
        )
    elif model_name == 'LightGBM':
        model = LGBMRegressor(
            n_estimators=100,
            learning_rate=0.1,
            num_leaves=31,
            random_state=42,
            n_jobs=-1,
            verbose=-1
        )
    else:
        model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
    
    # Time series cross-validation
    tscv = TimeSeriesSplit(n_splits=5)
    cv_scores = []
    
    for fold, (train_idx, val_idx) in enumerate(tscv.split(X_train)):
        print(f"  Fold {fold + 1}/5")
        X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
        
        # Train model
        model.fit(X_tr, y_tr)
        
        # Predict and evaluate
        y_pred = model.predict(X_val)
        mae = mean_absolute_error(y_val, y_pred)
        cv_scores.append(mae)
        
        print(f"    MAE: {mae:.2f}")
    
    avg_mae = np.mean(cv_scores)
    print(f"Average MAE across folds: {avg_mae:.2f}")
    
    return cv_scores

# Perform time-aware cross-validation
print("\n🔄 Time-aware Cross-Validation Results:")
xgb_cv_scores = time_aware_cross_validation(X_train, y_train, 'XGBoost')
lgb_cv_scores = time_aware_cross_validation(X_train, y_train, 'LightGBM')


## Step 8: Model Evaluation and Visualization

Evaluate all models and create comprehensive visualizations.


In [ ]:
print("📊 Evaluating models...")

results = {}

for model_name, model in models.items():
    print(f"Evaluating {model_name}...")
    
    # Make predictions
    y_pred = model.predict(X_test)
    
    # Calculate metrics
    mae = mean_absolute_error(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    r2 = r2_score(y_test, y_pred)
    
    results[model_name] = {
        'MAE': mae,
        'RMSE': rmse,
        'R2': r2,
        'predictions': y_pred
    }
    
    print(f"  MAE: {mae:.2f}")
    print(f"  RMSE: {rmse:.2f}")
    print(f"  R²: {r2:.4f}")


In [ ]:
# Create comprehensive visualizations
print("📈 Creating visualizations...")

# Set up the plotting style
fig, axes = plt.subplots(2, 2, figsize=(20, 15))
fig.suptitle('Sales Forecasting Results', fontsize=16, fontweight='bold')

# 1. Actual vs Predicted for all models
ax1 = axes[0, 0]
ax1.plot(test_dates, y_test, label='Actual', color='blue', linewidth=2)

colors = ['red', 'green', 'orange', 'purple']
for i, (model_name, result) in enumerate(results.items()):
    ax1.plot(test_dates, result['predictions'], 
            label=f'{model_name} (MAE: {result["MAE"]:.0f})', 
            color=colors[i], alpha=0.7, linewidth=1.5)

ax1.set_title('Actual vs Predicted Sales')
ax1.set_xlabel('Date')
ax1.set_ylabel('Weekly Sales')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Model comparison bar chart
ax2 = axes[0, 1]
model_names = list(results.keys())
mae_scores = [results[name]['MAE'] for name in model_names]
rmse_scores = [results[name]['RMSE'] for name in model_names]

x = np.arange(len(model_names))
width = 0.35

ax2.bar(x - width/2, mae_scores, width, label='MAE', alpha=0.8)
ax2.bar(x + width/2, rmse_scores, width, label='RMSE', alpha=0.8)

ax2.set_title('Model Performance Comparison')
ax2.set_xlabel('Models')
ax2.set_ylabel('Error')
ax2.set_xticks(x)
ax2.set_xticklabels(model_names, rotation=45)
ax2.legend()
ax2.grid(True, alpha=0.3)

# 3. Residuals plot for best model
best_model = min(results.keys(), key=lambda x: results[x]['MAE'])
ax3 = axes[1, 0]
residuals = y_test - results[best_model]['predictions']
ax3.scatter(results[best_model]['predictions'], residuals, alpha=0.6)
ax3.axhline(y=0, color='red', linestyle='--')
ax3.set_title(f'Residuals Plot - {best_model}')
ax3.set_xlabel('Predicted Values')
ax3.set_ylabel('Residuals')
ax3.grid(True, alpha=0.3)

# 4. Feature importance (if available)
ax4 = axes[1, 1]
if hasattr(models[best_model], 'feature_importances_'):
    importances = models[best_model].feature_importances_
    feature_names = feature_cols
    
    # Get top 10 features
    top_indices = np.argsort(importances)[-10:]
    top_features = [feature_names[i] for i in top_indices]
    top_importances = importances[top_indices]
    
    ax4.barh(range(len(top_features)), top_importances)
    ax4.set_yticks(range(len(top_features)))
    ax4.set_yticklabels(top_features)
    ax4.set_title(f'Top 10 Feature Importance - {best_model}')
    ax4.set_xlabel('Importance')
else:
    ax4.text(0.5, 0.5, f'{best_model}\nFeature importance\nnot available', 
            ha='center', va='center', transform=ax4.transAxes)
    ax4.set_title(f'Feature Importance - {best_model}')

plt.tight_layout()
plt.savefig('sales_forecasting_results.png', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# Print summary
print("\n" + "="*60)
print("📊 MODEL PERFORMANCE SUMMARY")
print("="*60)

for model_name, result in results.items():
    print(f"{model_name:15} | MAE: {result['MAE']:8.2f} | RMSE: {result['RMSE']:8.2f} | R²: {result['R2']:6.4f}")

best_model = min(results.keys(), key=lambda x: results[x]['MAE'])
print(f"\n🏆 Best Model: {best_model} (Lowest MAE: {results[best_model]['MAE']:.2f})")
print("="*60)

print("\n🎉 Sales Forecasting Project Completed!")
print("✅ All mandatory bonus features implemented:")
print("   - Rolling averages and statistical features")
print("   - Seasonal decomposition")
print("   - XGBoost and LightGBM models")
print("   - Time-aware cross-validation")
print("   - Comprehensive visualizations")
